<a href="https://colab.research.google.com/github/vlady98ish/Colab/blob/main/HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports



In [ ]:
import torch
from torch import nn
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#Init Linear Layer Class

In [ ]:
class Linear(torch.nn.Module):
  def __init__(self, in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None:
    factory_kwargs = {'device': device, 'dtype': dtype}
    super(Linear, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))
    if bias:
        self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()

  def reset_parameters(self) -> None:
    self.weight = nn.Parameter(torch.rand([self.out_features, self.in_features]))
    if self.bias is not None:
      self.bias = nn.Parameter(torch.rand([self.out_features]))

  def forward(self, input: torch.Tensor) -> torch.Tensor:
    return torch.matmul(input, torch.transpose(self.weight,0,1)) + self.bias

  def extra_repr(self) -> str:
    return 'in_features={}, out_features={}, bias={}'.format(
        self.in_features, self.out_features, self.bias is not None
      )

#Init BTU Class

In [ ]:
class BTU(torch.nn.Module):
  def __init__(self, T=0.2, inplace: bool = False):
      super(BTU, self).__init__()
      self.T = T

  def forward(self, input: torch.Tensor) -> torch.Tensor:
      return 1 / (1 + torch.exp(-input/self.T))

#Init XOR Model

In [ ]:
dim = 2
out_dim = 1


x_train = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], requires_grad=True, dtype=torch.float32)
print("X_Train tensor:",x_train)
t_train = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)
print("T_Train tensor:",t_train)
###MY CODE#####
x_validation = torch.tensor([[0, 0],[0, 1],[1, 0],[1, 1],[1,0.1],[1,0.9],[0.9,0.9],[0.1,0.9]], requires_grad=True, dtype=torch.float32)
print("X_Validation tensor:",x_validation)
t_validation = torch.tensor([[0], [1], [1], [0],[1],[0],[0],[1]])
print("T_Validation tensor:",t_validation)
###MY CODE#####

class XOR_Net_Model(nn.Module):
  def __init__(self,num_hidden, bypass=True):
    super().__init__()
    self.bypass = bypass
    self.hidden = Linear(dim, num_hidden)
    if self.bypass:
      self.output = Linear(num_hidden + dim, out_dim)
    else:
      self.output = Linear(num_hidden, out_dim)
    self.BTU = BTU(0.5)

  def forward(self, input):
    z1 = self.hidden(input)
    y1 = self.BTU(z1)
    if self.bypass:
      y1_concat = torch.cat((input, y1), 1)
      z2 = self.output(y1_concat)
    else:
      z2 = self.output(y1)
    return self.BTU(z2)

X_Train tensor: tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], requires_grad=True)
T_Train tensor: tensor([[0.],
        [1.],
        [1.],
        [0.]])
X_Validation tensor: tensor([[0.0000, 0.0000],
        [0.0000, 1.0000],
        [1.0000, 0.0000],
        [1.0000, 1.0000],
        [1.0000, 0.1000],
        [1.0000, 0.9000],
        [0.9000, 0.9000],
        [0.1000, 0.9000]], requires_grad=True)
T_Validation tensor: tensor([[0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1]])


#Init Loss Function

In [ ]:
def Loss(out, t_train):
  return -torch.sum(t_train * torch.log(out) + (1.0 - t_train) * torch.log(1.0 - out))  # Cross Entropy loss function

#Train Function

In [ ]:
def train(model, x_train, t_train, optimizer):
  y_pred = model(x_train)
  loss = Loss(y_pred, t_train)

  # zero gradients berfore running the backward pass
  optimizer.zero_grad()

  # backward pass to compute the gradient of loss
  # backprop + accumulate 
  loss.backward()

  # update params
  optimizer.step()
  return loss

#Test Function

In [ ]:
# define test step operation:
def test(model, x_test, t_test):
  loss = Loss(model(x_test), t_test)
  return loss

#MODEL RUN FUNCTION
10 runs

In [ ]:
def model_run10(learning_rate,hidden,bypass,epocs):
  good_training_count =0
  parameters_of_good_training =[]
  while(good_training_count!=10):
    model = XOR_Net_Model(hidden, bypass)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    counter_of_10 = 0
    previous_loss = 1000000
 
  
    for i in range(epocs):
      train_loss = train(model, x_train, t_train, optimizer)
      ###MY CODE#####
      
      validation_loss = test(model,x_validation,t_validation)
      difference = previous_loss - validation_loss
      # print("Diffetence of loss function: {}".format(difference))
      if difference <= 0.0001 and validation_loss<0.2:
        counter_of_10+=1
        print("COUNTER {}".format(counter_of_10))
      else:
        counter_of_10 = 0
      if counter_of_10 == 10:
        print("Good train")
        print("Validation Loss: {} ,Train Loss: {}, Count of epocs: {}".format(validation_loss,train_loss,i))
        results = {"Validation Loss":validation_loss,
                   "Training Loss": train_loss,
                 "count_of_epocs": i}
        parameters_of_good_training.append(results)
        good_training_count+=1
        break
      elif i == 40000:
        print("Bad train: number of epocs = {}".format(i))
        break  
      previous_loss = validation_loss
      ###MY CODE#####
      
      # print("\n Epoc: %s, train loss: %s" % (i, train_loss))
      # print("\n Epoc: %s, validation loss: %s" % (i, validation_loss))
      # print(model(x_train))
  print(parameters_of_good_training)

#Training

#<b>First Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = True


In [ ]:
l_rate = 0.1
num_hidden = 2
bypass = True
num_epocs = 1000000000

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

COUNTER 1
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.12576636672019958 ,Train Loss: 0.030804775655269623, Count of epocs: 1383
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.15816454589366913 ,Train Loss: 0.034310370683670044, Count of epocs: 1699


#<b>Second Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = False

In [ ]:
l_rate = 0.1
num_hidden = 2
bypass = False

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08499662578105927 ,Train Loss: 0.028669072315096855, Count of epocs: 1092
Bad train: number of epocs = 40000
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08512066304683685 ,Train Loss: 0.02858002856373787, Count of epocs: 1153
Bad train: number of epocs = 40000
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.0851076990365982 ,Train Loss: 0.02863696962594986, Count of epocs: 1095
Bad train: number of epocs = 40000
Bad train: number of epocs = 40000
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08515261858701706 ,Train Loss: 0.02865254133939743, Count of epocs: 1075
COUNTER 1
COUNTER 1
COUNTER 2
COUNTER 3
COUN

#<b>Third Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = True

In [ ]:
l_rate = 0.1
num_hidden = 4
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Fourth Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = False

In [ ]:
l_rate = 0.1
num_hidden = 4
bypass = False

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Fifth Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = True

In [ ]:
l_rate = 0.01
num_hidden = 2
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Six Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = False

In [ ]:
l_rate = 0.01
num_hidden = 2
bypass = False

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Seven Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = False

In [ ]:
l_rate = 0.01
num_hidden = 4
bypass = False

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Eight Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = True

In [ ]:
l_rate = 0.01
num_hidden = 4
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)
  

#<b>Nine Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 1<br>
3 - Bypass = True

In [ ]:
l_rate = 0.1
num_hidden = 1
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)